In [3]:
import json
from parsing.parsers import *
data = json.loads(open('piecewise.json').read())

for curve in data['curves']:
    config = curve['curveConfig']
    for rateHelper in config['rateHelpers']:
        values = parse(**rateHelper['helperConfig'])
        print(values)

{'dayCounter': <ORE.ORE.Actual360; proxy of <Swig Object of type 'QuantLib::Actual360 *' at 0x10403d4a0> >, 'tenor': Period("1D"), 'calendar': <ORE.ORE.NullCalendar; proxy of <Swig Object of type 'QuantLib::NullCalendar *' at 0x10403e340> >, 'settlementDays': 0, 'endOfMonth': False, 'convention': 4}
{'dayCounter': <ORE.ORE.Actual360; proxy of <Swig Object of type 'QuantLib::Actual360 *' at 0x103f10270> >, 'tenor': Period("2M"), 'calendar': <ORE.ORE.NullCalendar; proxy of <Swig Object of type 'QuantLib::NullCalendar *' at 0x103f12dc0> >, 'settlementDays': 0, 'endOfMonth': False, 'convention': 4}
{'dayCounter': <ORE.ORE.Actual360; proxy of <Swig Object of type 'QuantLib::Actual360 *' at 0x10403c9c0> >, 'tenor': Period("3M"), 'calendar': <ORE.ORE.NullCalendar; proxy of <Swig Object of type 'QuantLib::NullCalendar *' at 0x10403def0> >, 'settlementDays': 0, 'endOfMonth': False, 'convention': 4}
{'dayCounter': <ORE.ORE.Actual360; proxy of <Swig Object of type 'QuantLib::Actual360 *' at 0x103

In [36]:
from parsing.enums import *
from collections import deque


def getDependencyList(data: dict) -> dict:
    # possible curve related keys
    pc = ['discountCurve', 'discountingCurve', 'collateralCurve']
    # possible index related keys
    pi = ['index', 'shortIndex', 'longIndex']

    dependecies = {}
    for curve in data['curves']:
        curveName = curve['curveName']
        if curveName not in dependecies.keys():
            dependecies[curveName] = set()

        curveConfig = curve['curveConfig']
        curveType = CurveType(curveConfig['curveType'])
        if curveType == CurveType.Piecewise:
            for rateHelper in curveConfig['rateHelpers']:
                config = rateHelper['helperConfig']
                for key in pc:
                    if key in config.keys():
                        dependecies[curveName].add(config[key])
                for key in pi:
                    if key in config.keys():
                        dependecies[curveName].add(config[key])
    return dependecies


dependencies = getDependencyList(file)
dependencies


{'ICP': {'CLP_COLLUSD', 'ICP'},
 'SOFR': {'SOFR'},
 'UF_COLLUSD': {'CLP_COLLUSD', 'ICP'},
 'CLP_COLLUSD': {'SOFR'},
 'LIBOR1M': {'LIBOR1M', 'LIBOR3M', 'SOFR'},
 'LIBOR3M': {'LIBOR3M', 'SOFR'},
 'LIBOR6M': {'LIBOR3M', 'LIBOR6M', 'SOFR'},
 'LIBOR12M': {'LIBOR12M', 'LIBOR3M', 'SOFR'}}

In [33]:
def topologicalSort(dependencies):

    for element, deps in dependencies.items():
        deps.discard(element)

    # Find elements with no dependencies
    noDependency = deque([k for k, v in dependencies.items() if not v])

    sortedElements = []

    while noDependency:
        currentElement = noDependency.popleft()
        sortedElements.append(currentElement)

        # Remove the current element as a dependency from other elements
        for element, deps in dependencies.items():
            if currentElement in deps:
                deps.remove(currentElement)
                # If the element now has no dependencies, add it to the queue
                if not deps and element not in noDependency:
                    noDependency.append(element)

    return sortedElements


In [34]:
sortedList = topologicalSort(dependencies)


['SOFR',
 'CLP_COLLUSD',
 'LIBOR3M',
 'ICP',
 'LIBOR1M',
 'LIBOR6M',
 'LIBOR12M',
 'UF_COLLUSD']